In [96]:
import six

In [97]:
DATA = """
mask = 000000000000000000000000000000X1001X
mem[42] = 100
mask = 00000000000000000000000000000000X0XX
mem[26] = 1
""".strip()

In [98]:
def binaryStringToNumericDecimal(s):
    """
    Converts the given (string'd) binary number to the
    equivalent (int'd) decimal number.
    
    :param s: str
    :return: int
    """
    return int(s, 2)

In [99]:
def numericDecimalToBinaryString(n, pad=None):
    """
    Converts the given (int'd) decimal number to the
    equivalent (string'd) binary number. Optionally,
    the output can be (zero) padded.
    
    :param n: int
    :param pad: int
    :return: s
    """
    s = str(bin(n))[2:]
    if pad: s = s.rjust(pad, '0')
    return s

In [100]:
def _execute1(instructions):
    
    """
    Given a memory instructions and a mask, it proceeds to apply the
    instructions and to keep track of the memory status, which is
    eventually returned along with the sum of all values in memory.
    
    In this mode of execution, the mask specifies a transformation
    to be applied to the contents stored in memory.
    
    :param instructions: list(str,)
    :return: memory:dict(address:int, content:int), contentSum:int
    """
    
    mask, memory = {}, {}
    
    for line in instructions:
        # For every instruction line, extract the command name and its argument;
        command, content = tuple(line.split(' = '))
        if command == 'mask':
            # If the instruction is an update to the mask, store the mapping
            # between the bit index and its overwriting value;
            mask = {i: c for i, c in enumerate(content) if c in ('0', '1')}
        elif command.startswith('mem'):
            # If the instruction is an update or addition to memory, first
            # extract the memory location;
            address = int(command.split('[')[1].split(']')[0])
            # Then extract the value to be stored (as a decimal integer);
            value = int(content)
            # Then cast it to 36-bit long binary string;
            value = numericDecimalToBinaryString(value, pad=36)
            # Update the bits based on the mask;
            value = {i: mask.get(i, c) for i, c in enumerate(value)}
            # Then merge it back to a string;
            value = ''.join([value[i] for i in range(36)])
            # And cast it back to a decimal integer to be stored;
            value = binaryStringToNumericDecimal(value)
            memory[address] = value
    
    return memory, sum(six.itervalues(memory))

In [101]:
def _execute2(instructions):
    
    """
    Given a memory instructions and a mask, it proceeds to apply the
    instructions and to keep track of the memory status, which is
    eventually returned along with the sum of all values in memory.
    
    In this mode of execution, the mask specifies a transformation
    to be applied to the memory address.
    
    :param instructions: list(str,)
    :return: memory:dict(address:int, content:int), contentSum:int
    """
    
    mask, memory = {}, {}
    
    for line in instructions:
        # For every instruction line, extract the command name and its argument;
        command, content = tuple(line.split(' = '))
        if command == 'mask':
            # If the instruction is an update to the mask, store the mapping
            # between the bit index and its operation;
            mask = {i: c for i, c in enumerate(content)}
        elif command.startswith('mem'):
            # If the instruction is an update or addition to memory, first
            # extract the memory location;
            address = int(command.split('[')[1].split(']')[0])
            # Cast the decimal integer address to a binary string;
            address = numericDecimalToBinaryString(address, pad=36)
            # Then generate a mapping between the index of each bit in the
            # address and its value;            
            address = {i: c for i, c in enumerate(address)}
            # Apply the mask to the address, preserving every address bit
            # for which the mask bit is 0, else update it to be the value
            # in the mask;
            for i, c in six.iteritems(mask):
                address[i] = address[i] if c == '0' else c
            # Merge the address back to a binary string;
            address = ''.join([address[i] for i in range(36)])
            # Evaluate the new address to determine all the real addresses
            # to which it maps, based on the X values added by the mask;
            if 'X' in address:
                # Track all 'floating' address bits;
                addresses, floating = [], [i for i, c in enumerate(address) if c == 'X']
                # For as many times as there are permutations of the floating values;
                for j in range(2 ** len(floating)):
                    # Convert the binary string address to a list of binary characters;
                    a = [c for c in address]
                    # Determine the binary values to be filled in by converting
                    # the permutation iteration index to a binary string;
                    evaluated = numericDecimalToBinaryString(j, len(floating))
                    # Replace the floating values with the values in the determined
                    # binary value of the permutation iteration index;
                    for l, k in enumerate(floating):
                        a[k] = evaluated[l]
                    # Merge back the address to a binary string and add it to the list
                    # of addresses to be updated;
                    addresses.append(''.join(a))
            else:
                # If there are no floating bits in the address, then the address us the
                # only one to be updated;
                addresses = [address]
            
            # Update all found addressed with the operation argument value;
            for address in addresses:
                memory[binaryStringToNumericDecimal(address)] = int(content)

    return memory, sum(six.itervalues(memory))

In [102]:
def execute(mode, instructions):
    if mode == 1: return _execute1(instructions)
    elif mode == 2: return _execute2(instructions)
    else: raise NotImplementedError('Only execution modes 1 and 2 are available')

In [103]:
instructions = DATA.splitlines()

In [104]:
_, firstResult = execute(1, instructions)
_, secondResult = execute(2, instructions)
firstResult, secondResult

(51, 208)